In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/My\ Drive/Lastsemester/Social_Network_Analysis/project

Mounted at /content/gdrive
/content/gdrive/My Drive/Lastsemester/Social_Network_Analysis/project


In [2]:
# Importing modules
import pandas as pd
import numpy as np

In [ ]:
! pip install transformers

from transformers import AutoModel
from transformers import AutoTokenizer
import torch

model_ckpt = "distilbert-base-uncased"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModel.from_pretrained(model_ckpt).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

# Practice Dataset

In [140]:
d = {'product_id': ['B0188A3QRM', 'B075VXJ9VG'], 
     'product_title': ['Amazon Basics Woodcased #2 Pencils, Unsharpened', 'BAZIC Pencil #2 HB Pencils, Latex Free Eraser'],
     'product_description': [np.NaN, '<p><strong>BACK TO BAZIC</strong></p><p>Our go'], 
     'product_bullet_point': ['144 woodcase #2 HB pencils made from high-quality', '&#11088; UN-SHARPENED #2 PREMIUM PENCILS.'],
     'product_brand': ['Amazon Basics	', 'BAZIC Products	'],
     'product_color_name': [ 'Yellow', '12-count'],
     'product_locale': [ 'us', 'us']}
df = pd.DataFrame(data=d)

In [141]:
# add ".astype(str)" 
df['Combined'] = df['product_title'].astype(str) + ', ' + df['product_description'].astype(str) + ', ' + df['product_bullet_point'].astype(str)  + ', ' + df['product_brand'].astype(str) + ', ' + df['product_color_name'].astype(str) + ', ' + df['product_locale'].astype(str)
df = pd.DataFrame(data=df)
df



,product_id,product_title,product_description,product_bullet_point,product_brand,product_color_name,product_locale,Combined
0,B0188A3QRM,"Amazon Basics Woodcased #2 Pencils, Unsharpened",NaN,144 woodcase #2 HB pencils made from high-quality,Amazon Basics\t,Yellow,us,"Amazon Basics Woodcased #2 Pencils, Unsharpene..."
1,B075VXJ9VG,"BAZIC Pencil #2 HB Pencils, Latex Free Eraser",<p><strong>BACK TO BAZIC</strong></p><p>Our go,&#11088; UN-SHARPENED #2 PREMIUM PENCILS.,BAZIC Products\t,12-count,us,"BAZIC Pencil #2 HB Pencils, Latex Free Eraser,..."


# Breaking up each entry to lists of 512 max

Breaks up each entry in the "Combined" column to sets of 50 (change to 512)

In [142]:
broken_text = []
s = df['Combined']
for text in s:
  n = 50
  a = [text[i:i+n] for i in range(0, len(text), n)]
  broken_text.append(a)
#broken_text
df

,product_id,product_title,product_description,product_bullet_point,product_brand,product_color_name,product_locale,Combined
0,B0188A3QRM,"Amazon Basics Woodcased #2 Pencils, Unsharpened",NaN,144 woodcase #2 HB pencils made from high-quality,Amazon Basics\t,Yellow,us,"Amazon Basics Woodcased #2 Pencils, Unsharpene..."
1,B075VXJ9VG,"BAZIC Pencil #2 HB Pencils, Latex Free Eraser",<p><strong>BACK TO BAZIC</strong></p><p>Our go,&#11088; UN-SHARPENED #2 PREMIUM PENCILS.,BAZIC Products\t,12-count,us,"BAZIC Pencil #2 HB Pencils, Latex Free Eraser,..."


Makes a new column in df to place the String Splits

In [143]:
df['Split_String'] = ""


Inputting the values

In [144]:
X = df["Split_String"]

i = 0
while i < len(X): #change 10 to length of dataframe
  X[i] = broken_text[i]
  i+=1
df


,product_id,product_title,product_description,product_bullet_point,product_brand,product_color_name,product_locale,Combined,Split_String
0,B0188A3QRM,"Amazon Basics Woodcased #2 Pencils, Unsharpened",NaN,144 woodcase #2 HB pencils made from high-quality,Amazon Basics\t,Yellow,us,"Amazon Basics Woodcased #2 Pencils, Unsharpene...","[Amazon Basics Woodcased #2 Pencils, Unsharpen..."
1,B075VXJ9VG,"BAZIC Pencil #2 HB Pencils, Latex Free Eraser",<p><strong>BACK TO BAZIC</strong></p><p>Our go,&#11088; UN-SHARPENED #2 PREMIUM PENCILS.,BAZIC Products\t,12-count,us,"BAZIC Pencil #2 HB Pencils, Latex Free Eraser,...","[BAZIC Pencil #2 HB Pencils, Latex Free Eraser..."


Getting Bert Numbers

In [145]:
df['BERT Numbers'] = ""

In [146]:
X = df["Split_String"]
Bert = df['BERT Numbers']

i=0
X = df["Split_String"]
while i < len(X):
  embeddings=[]
  for o in X[i]:
    text = o
    inputs = tokenizer(text, return_tensors="pt")
    inputs = {k:v.to(device) for k,v in inputs.items()}
    with torch.no_grad():
      outputs = model(**inputs)
    embeddings.append(outputs.last_hidden_state)
  Bert[i]= embeddings  
  i+=1

In [147]:
y = df['BERT Numbers']

In [117]:
y

0    [[[tensor([-4.2097e-01, -3.5455e-01, -4.3772e-...
1    [[[tensor([-3.0958e-01, -1.2243e-01, -1.3312e-...
Name: BERT Numbers, dtype: object

Each tensor matrix is different # of rows, but 768 columns. 

In [148]:
df['Mean of Bert'] = ""

In [165]:
Means = df['Mean of Bert']
i=0
Averages = []
while i < len(y): # num of rows in data set (y is the column of bert nums)
  concated_means = []
  j=0
  while j < len(y[i]): # num of bert numbers per product. (sets of 512)
    concated_means.append(torch.mean(y[i][j], dim=1))
    j+=1
  y_concats = torch.cat(concated_means)
  y_avg = torch.mean(y_concats, dim = 0)
  Means[i]= y_avg
  i+=1


In [ ]:
y1=torch.mean(y[0][0], dim=1)
y2=torch.mean(y[0][1], dim=1)
y3=torch.mean(y[0][2], dim=1)

y_avg = torch.cat([y1,y2,y3])
y_avg = torch.mean(y_avg, dim = 0)


# Run This Below on Real DataSet

In [ ]:
## STEP 1
# First create column for concatting all description columns for products
df['Combined'] = df['product_title'].astype(str) + ', ' + df['product_description'].astype(str) + ', ' + df['product_bullet_point'].astype(str)  + ', ' + df['product_brand'].astype(str) + ', ' + df['product_color_name'].astype(str) + ', ' + df['product_locale'].astype(str)
df = pd.DataFrame(data=df)

## STEP 2
# Then break up each entry in the "Combined" column into sets of 512 characters
df['Split_String'] = ""
broken_text = []
s = df['Combined']
for text in s:
  n = 512
  a = [text[i:i+n] for i in range(0, len(text), n)]
  broken_text.append(a)

## STEP 3
# Then i put it into another column "Split_String"(idk why) but in list form, so i can loop thru it
X = df["Split_String"]
i = 0
while i < len(X): #change 10 to length of dataframe
  X[i] = broken_text[i]
  i+=1

## STEP 4: 
# calculating bert numbers for each set of 512 sets
df['BERT Numbers'] = ""
X = df["Split_String"]
Bert = df['BERT Numbers']

i=0
X = df["Split_String"]
while i < len(X):
  embeddings=[]
  for o in X[i]:
    text = o
    inputs = tokenizer(text, return_tensors="pt")
    inputs = {k:v.to(device) for k,v in inputs.items()}
    with torch.no_grad():
      outputs = model(**inputs)
    embeddings.append(outputs.last_hidden_state)
  Bert[i]= embeddings  
  i+=1


## STEP 5:
# getting the averages and storing them in "Mean of Bert"
y = df['BERT Numbers']
df['Mean of Bert'] = ""

Means = df['Mean of Bert']
i=0
Averages = []
while i < len(y): # num of rows in data set (y is the column of bert nums)
  concated_means = []
  j=0
  while j < len(y[i]): # num of bert numbers per product. (sets of 512)
    concated_means.append(torch.mean(y[i][j], dim=1))
    j+=1
  y_concats = torch.cat(concated_means)
  y_avg = torch.mean(y_concats, dim = 0)
  Means[i]= y_avg
  i+=1




